<a href="https://colab.research.google.com/github/nomomon/chess-bot-competition/blob/main/practical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cover Chess Bot Competition

Welcome to the very first Cover Chess Bot competition.

## Setup

Let's install the [`python-chess`](https://python-chess.readthedocs.io/en/latest/) library, it will be used for managing the chess board, as well as the competition. This library provides an extensive API of controlling the chess game.

Visit their website for more documentation.

In [ ]:
%%capture
!pip install python-chess

In [ ]:
import numpy as np
import pandas as pd

import chess

## Introduction

Try running the example bellow, which demonstrates how to make moves in this library.

In [ ]:
import chess

# make a board (new game)
board = chess.Board()

# do a step (this is in `san` format)
board.push_san("Nh3")

# visualise the board
display(board)

## Setup

Throught the practical as well as in the competition we will be defining the bots using this class. It recieves a [`board_fen`](https://en.wikipedia.org/wiki/Forsyth%E2%80%93Edwards_Notation), which is a string represntation of a chess board.

In [ ]:
from abc import ABC, abstractmethod


class ChessBotClass(ABC):
    @abstractmethod
    def __call__(self, board_fen: str) -> chess.Move:
        pass

Also we will have a `Judge`, which will manage the competition.

In [ ]:
from IPython.display import clear_output
from itertools import count
import time



class Judge():
    def __init__(self, player_1, player_2, time_limit=300):
        self.player_1 = player_1
        self.player_2 = player_1
        self.time_limit = time_limit

    def run_game(self, initial_board_fen:str = None):
        board = chess.Board()

        player_times = [0, 0]

        for i in count(0, 1):
            if board.is_checkmate():
                print("GAME OVER")
                print("Winner is bot", i%2 + 1, sep="_")
                break
            if i > 100:
                print("GAME OVER")
                print("Exceeded move limits, it's a tie")
                break

            board_fen = board.fen()

            start = time.time()
            if i % 2 == 0:
                move = self.player_1(board_fen)
            else:
                move = self.player_2(board_fen)
            end = time.time()

            player_times[i%2] += end - start

            if player_times[i%2] > self.time_limit:
                print("GAME OVER")
                print("Time limit exceeded, winner is bot", i%2+1, sep="_")

            if not board.is_legal(move):
                raise ValueError("Illegal board move. The bot it hallucinating...")

            board.push(move)

            clear_output(wait=True)
            display(board)

            # slow down the bots so that we can see them
            time.sleep(.25)
        print("Times used:", player_times)

## Bots

### Random Bots

Let's make a sample bot, which recives the board state (in fen format) and does a random legal move.

In [ ]:
import random

class RandomBot(ChessBotClass):
    def __call__(self, board_fen):
        board = chess.Board(board_fen)

        moves = list(board.legal_moves)

        idx = int(random.random() * len(moves))

        move = moves[idx]

        return move

Now, let's initialize two random bots and run a simple tournament between them :D

In [ ]:
# initialize the bots
bot_1 = RandomBot()
bot_2 = RandomBot()

# run tournament
judge = Judge(bot_1, bot_2)
judge.run_game()

### MiniMax

In [ ]:
INF = 1e10

class MiniMaxBot(ChessBotClass):
    def __init__(self, max_depth: int) -> None:
        self._max_depth = max_depth

    def evaluate_board(self, board, turn):
        return 0

    def minimax(self, depth: int,
                board: chess.Board,
                alpha: float, beta: float,
                maximizing: bool) -> float:

        if depth == 0 or board.is_game_over():
            return self.evaluate_board(board, board.turn) * -1**(not maximizing)
        if maximizing:
            value = -INF
            for move in board.legal_moves:
                board.push(move) #also switches turn
                value = max(value, self.minimax(depth - 1,
                                                board, alpha,
                                                beta, not maximizing))
                board.pop()
                if value > beta:
                    break
                alpha = max(alpha, value)
        else:
            value = INF
            for move in board.legal_moves:
                board.push(move)
                value = min(value, self.minimax(depth - 1,
                                           board, alpha,
                                           beta, not maximizing))
                board.pop()
                if value < alpha:
                    break
                beta = min(beta, value)
        return value

    def __call__(self, board_fen: str) -> chess.Move:
        board = chess.Board(board_fen)

        best_move = None
        best_eval = -INF
        for move in board.legal_moves:
            board.push(move)
            value = self.minimax(self._max_depth, board, -INF, INF, True)
            if value > best_eval:
                best_eval = value
                best_move = move
            board.pop()
        return best_move

In [ ]:
# initialize the bots
bot_1 = MiniMaxBot(3)
bot_2 = RandomBot()

# run tournament
judge = Judge(bot_1, bot_2)
judge.run_game()

### Piece Number Bot



In [ ]:
class NumPieceBot(MiniMaxBot):
    def evaluate_board(self, board: chess.Board, color: chess.Color) -> float:
        return sum([piece.color == color for piece in board.piece_map().values()])

In [ ]:
# initialize the bots
bot_1 = MiniMaxBot(3)
bot_2 = NumPieceBot(3)

# run tournament
judge = Judge(bot_1, bot_2)
judge.run_game()

### Piece Value Bot

In [ ]:
DEFAULT_PIECE_VALUES = {chess.PAWN : 1, chess.ROOK : 5, chess.KNIGHT : 3, chess.BISHOP : 3, chess.KING : 100, chess.QUEEN : 9}

class PieceValueBot(MiniMaxBot):
    def __init__(self,
            max_depth: int,
            piece_values: dict = DEFAULT_PIECE_VALUES
        ) -> None:
        super().__init__(max_depth)
        self.piece_values = piece_values

    def evaluate_board(self, board: chess.Board, color: chess.Color) -> float:
        return sum([self.piece_values[piece.piece_type] for piece in board.piece_map().values() if piece.color == color])

In [ ]:
# initialize the bots
bot_1 = NumPieceBot(3)
bot_2 = PieceValueBot(3)

# run tournament
judge = Judge(bot_1, bot_2)
judge.run_game()

# What's next?

- Develop your own bots
- Research on different bots in chess
    - See a recent chess bot competition for inspiration
    - https://www.youtube.com/watch?v=Ne40a5LkK6A
- Submit your own bot and see who's bot is the best!
    - Sign in through University account
    - https://forms.gle/jszsX8SvETQWobkP6
- The competition will be ran everyday at midnight and the results will be updated
    - Starting `Fri 9 Feb 2024 00:00`
    - See them [here](https://fully-connected-graph.github.io/chess-bot-competition/)